In [1]:
import scipy.integrate as integrate
import numpy as np
import matplotlib.pyplot as plt

#-------------- Legend ---------------

sigma_xs = np.arange(1,10,0.5)
sigma_zs = np.arange(1,10,0.5)
sigma_x = 3.
sigma_z = 3.
phi = np.pi/2.
x_e = 0.584
x_es = np.linspace(0, 21.5, 500)
r_a = 0.01
r_as = np.linspace(0.01, 0.2, 6)
r_lower = 0
r_upper = r_a
theta_lower = 0
theta_upper = 2*np.pi

prefactor = 0.

earth_r = 6371
orbit_height = 500
orbit_r = (earth_r + orbit_height)*10**3
G = 6.67408 * 10**-11  # Gravitational constant
M = 5.972 * 10**24     # Mass of the Earth
T = np.sqrt(4*np.pi**2 * orbit_r**3 / (G*M)) # orbit period
N = 1000 # number of satellites in the cluster
S = 2*np.pi*orbit_r / N  # spacing between the satellites in the cluster

#-------------------------------------

print(T)

def P(r,theta):
  
  return r * np.exp(-0.5*r**2 * ((np.sin(theta) / sigma_x)**2 + (np.cos(theta) / sigma_z)**2 ) + r*x_e * ( (np.sin(phi) * np.sin(theta)/ sigma_x**2) + (np.cos(phi) * np.cos(theta) / sigma_z**2) ) )
  
    
    
def prob_collision(xe,phi):

    prefactor = np.exp(-0.5 * x_e**2 * ( (np.sin(phi) / sigma_x)**2 + (np.cos(phi) / sigma_z)**2 ) )/(2*np.pi * sigma_x * sigma_z)
    temp = prefactor * integrate.dblquad(P, r_lower, r_a, theta_lower, theta_upper)[0]
    return temp

5668.31775008775


In [2]:
def probability_col_N_iters(Xe_init, phi_init, N):  # expected values of closest distance and angle between the two satellites
    Xe = Xe_init
    phi = phi_init
    p_sur_N  = 1
    sigma_Xe = 0.550
    sigma_phi= np.pi*5/180 # radians
    for i in range(1,2*N+1):  # where N is the number of orbits (2 crossing points for each)
        p_col = prob_collision(Xe,phi)
        p_sur = 1 - p_col
        p_sur_N = p_sur_N * p_sur
        Xe = np.random.normal(Xe_init,sigma_Xe)
        phi = np.random.normal(phi_init,sigma_phi)
        
    return p_sur_N


In [3]:
def exp_num_orb_bef_crash(Xe_init, phi_init, Nmax):  # Expected number of Orbits before crash 
    Xe = Xe_init
    phi = phi_init
    p_sur_N  = 1
    sigma_Xe = 0.550
    sigma_phi= np.pi*5/180 # radians
    summ = 0
    for i in range(1,2*Nmax+1):  # where N is the number of orbits (2 crossing points for each)
        p_col = prob_collision(Xe,phi)
        summ = summ + i * p_col * p_sur_N
        p_sur = 1 - p_col
        p_sur_N = p_sur_N * p_sur
        Xe = np.random.normal(Xe_init,sigma_Xe)
        phi = np.random.normal(phi_init,sigma_phi)
    return summ

In [5]:
exp_num_orb_bef_crash(x_e,phi,100)

23.26743201544942

In [6]:
exp_num_orb_bef_crash(x_e,phi,1000)

551.1973181569679